# Lecture 10: The Expected Value of Random Variables
***

We'll need Numpy, Matplotlib, and possibly Pandas for this notebook, so let's load them. 

In [3]:
import numpy as np 
import matplotlib.pylab as plt 
import pandas as pd 
%matplotlib inline

### Exercise 1 - Chuck-a-luck 
*** 

The game **Chuck-a-luck** (found in fairgrounds) is played by rolling 3 dice and betting
on a number between 1 and 6. You win your bet multiplied by the number of times your
chosen number appears on the the three dice. You lose your bet if your number doesn't appear at all. For example 

- If you bet $\$1$ on $5$ and you roll $\{3,~4,~5\}$ then you get to keep your $\$1$ plus you win another $\$1$. 
- If you bet $\$1$ on $5$ and roll $\{4,~5,~5\}$ you get to keep your $\$1$ plus you win $\$2$. 
- On the other hand, if you bet $\$1$ on $5$ and roll $\{2,~3,~4\}$ then you lose your $\$1$. 

A quick look at this game may make it appear reasonably fair. Since you roll 3
dice and there seems to be a probability $\frac{1}{2}$ that your chosen number appears and so the
odds should be in your favor.

**Part A**: Let's write a function called chuck_a_trial that takes as its sole required argument the integer my_number that you bet on and returns your winnings or losses. To make things simple, we'll assume that we always bet $\$1$ on every roll.  If you like, you can add an optional verbose flag for printing helpful output. 

**Solution**: 

In [56]:
def chuck_a_trial(my_number, verbose=False):
    dice = np.array([1,2,3,4,5,6])
    rolls = np.random.choice(dice, size=3)
    winnings = np.sum(rolls == my_number)
    net = winnings if winnings > 0 else -1 
    if verbose: 
        print("rolled = {}, netted ${}".format(rolls, net))
    return net
    
chuck_a_trial(5, verbose=True);

rolled = [2 2 1], netted $-1


**Part B**: Write a function chuck_a_sim that takes the integer my_number that you bet on and runs many simulations of chuck_a_trial and computes your average winnings over all of the trials.  To control the number of trials in your simulation, add an optional parameter num_trials initialized to 1000. 

**Solution**: 

In [57]:
def chuck_a_sim(my_number, num_trials=int(1e3)):
    winnings = np.array([chuck_a_trial(my_number) for ii in range(num_trials)])
    return np.sum(winnings) / num_trials 

print("E[Winnings] = {:.3f}".format(chuck_a_sim(5, num_trials=int(1e5))))

E[Winnings] = -0.078


**Part C**: Based on your simulation above, how fair or unfair is this game?  Or said another way, how much do you expect to win/lose if you play this game for a very long time? 

**Solution**: From the experiment it appears that we expect to lose around 8 cents on average for every dollar that we bet. 

<br>

### Exercise 2 - Simulating Roulette
*** 

A Las Vegas roulette board contains 38 numbers $\{0, 00, 1, 2, \ldots, 36\}$. Of the non-zero numbers, 18 are red and 18 are black. You can place bets on various number/color combinations and each type of bet pays-out at a different rate.  For example: 

- If you bet $\$1$ on red (or black) and win you win $\$1$. 
- If you bet any particular number and win you win $\$35$. 
- If you bet on the first dozen (or second dozen, or third dozen) nonzero numbers and win you win $\$2$. 

![alt text](https://www.lasvegasdirect.com/wp-content/uploads/2016/09/American-Roulette-Table.gif)

It seems like there are so many ways to win!  In reality, some very careful probability theory was done by the game designers to ensure that there is not much difference in any particular payout.  We'll explore roulette both by simulation and by hand in this exercise. 

The following function simulates the spin of a Las Vegas roulette board.  

In [42]:
def spin_roulette():
    numbers = np.array(["0", "00"] + [str(ii) for ii in range(1,36+1)])
    red = [str(ii) for ii in [1,3,5,7,9,12,14,16,18,19,21,23,25,27,30,32,34,36]] 
    black = [str(ii) for ii in [2,4,6,8,10,11,13,15,17,20,22,24,26,29,29,31,33,35]]
    green = ["0", "00"]
    number = np.random.choice(numbers)
    color = "R" if number in red else "B" if number in black else "G"
    return number+color

spin_roulette()

'3R'

**Part A**: Write a function that estimates the expected winnings by betting on red (or black)

In [36]:
def color_winnings(color="R", num_spins=int(1e3)):
    num_winners = np.sum([color in spin_roulette() for ii in range(num_spins)])
    num_losers = num_spins - num_winners
    return (1*num_winners - 1*num_losers)/num_spins
    
print("E[Winings] = {:.3f}".format(color_winnings(color="R", num_spins=int(1e5))))

E[Winings] = -0.055


**Part B**: Write a function that estimates the expected winnings by betting on a particular number. 

In [39]:
def number_winnings(number="00", num_spins=int(1e3)):
    num_winners = np.sum([number in spin_roulette() for ii in range(num_spins)])
    num_losers = num_spins - num_winners
    return (35*num_winners - 1*num_losers)/num_spins
    
print("E[Winings] = {:.3f}".format(number_winnings(number="00", num_spins=int(1e6))))

E[Winings] = -0.052


**Part C**: Write a function that estimates the expected winnings by betting on the first dozen nonzero numbers. 

In [45]:
def dozen_winnings(num_spins=int(1e3)):
    first_dozen = [str(ii) for ii in range(1,12+1)]
    spins = [spin_roulette() for ii in range(num_spins)]
    num_winners = np.sum([spin[:-1] in first_dozen for spin in spins])
    num_losers = num_spins - num_winners
    return (2*num_winners - 1*num_losers)/num_spins
    
print("E[Winings] = {:.3f}".format(dozen_winnings(num_spins=int(5e5))))

E[Winings] = -0.052


**Part D**: Do you see a pattern?! Let's see if we can confirm our hypothesis.  First, compute the exact expectation of your winnings by betting on red. 

**Solution**: There are 18 red numbers and 20 non-red numbers.  Thus we have 

$$
E[\textrm{Winnings w/ Red}] = 1\cdot\frac{18}{38} - 1 \cdot \frac{20}{38} =  -\frac{2}{38}
$$

**Part E**: Now compute the exact expectation of your winnings by betting on any particular number. Remember that the payout for hitting a number is $35$ to $1$.  

**Solution**: The chance of getting a particular number is $1$ in $38$.  We have: 

$$
E[\textrm{Winnings for Single Number}] = 35\cdot\frac{1}{38} - 1 \cdot \frac{37}{38} =  -\frac{2}{38}
$$

**Part F**: Finally, compute the expected winnings by betting on any dozen nonzero numbers.  Remember that the payout for hitting the correct dozen is 2 to 1.  

**Solution**: The chance of getting in the correct dozen is $12$ in $38$.  We have: 

$$
E[\textrm{Winnings for Single Number}] = 2\cdot\frac{12}{38} - 1 \cdot \frac{26}{38} =  -\frac{2}{38}
$$

**Part G**: OK, so you're hopefully onto the pattern by now.  Based on the computations done above, can you come up with the odds function $f(n)$ that describes the payout odds for landing on 1 out of $n$ squares? 

**Solution**: We want the expected payout for a winning bet to always be $-\dfrac{2}{38}$. In other words, we want  

$$
f(n) \cdot \frac{n}{38} - 1 \cdot \frac{38-n}{38} = - \frac{2}{38}
$$

Solving for $f(n)$ we obtain 

$$
f(n) = \frac{36}{n}-1
$$

## Exercise 3 - The Expected Triangle
*** 

Suppose we choose arbitrarily a point from the square with corners at $(0,0)$, $(1,0)$, $(1,1)$, and $(0,1)$. Let the random variable $A$ be the area of the triangle with its corners at $(0,0)$, $(1,0)$, and the chosen point. 

![alt text](https://www.cs.colorado.edu/~ketelsen/files/courses/csci3022/figs/triangle.png)

**Part A**: Compute by hand the expected area of the triangle, $E[A]$. 

**Solution**: The important thing to notice is that since $A = \frac{1}{2} \times \textrm{base} \times \textrm{height}$ the area of the triangle only depends on the $y$-value of it's coordinates. Furthermore, we know that the base is always 1, so we have $A = \frac{1}{2}\times \textrm{height}$.  

Let $Y$ be a uniform random variable from $U[0,1]$.  Then $Y$ has probability density function 

$$
f(y) = \left\{ 
\begin{array}{cl}
1 & 0 \leq y \leq 1 \\
0 & \textrm{otherwise}
\end{array}
\right.
$$

We have by linearity of expectation that 

$$
E[A] = E[\frac{1}{2}Y] = \frac{1}{2}E[Y] = \frac{1}{2}\int_0^1 y\cdot 1~dy = \frac{1}{2}\cdot\left. \frac{y^2}{2}\right|_{~0}^{~1} = \frac{1}{2} \cdot \frac{1}{2} = \frac{1}{4}
$$

**Part B**: Write a brief simulation to confirm the by-hand computation you did in **Part A**. 

In [52]:
def triangle_sim(num_samples=int(1e3)):
    return np.sum(.5*np.random.uniform(0,1,size=num_samples))/num_samples

print("E[A] \u2248 {:.4f}".format(triangle_sim(num_samples=int(1e6))))

E[A] ≈ 0.2498


## Exercise 4 - Rising Temperatures
*** 

Let $X$ be a random variable describing the temperature in degrees Celsius in a town in the Southwester United States with probability density function 

$$
f(x) = \left\{
\begin{array}{cl}
\frac{2}{5}\left[\left(\frac{x}{50}\right)^3 - \left(\frac{x}{50}\right)^4 \right] & 0 \leq x \leq 50 \\
0 & \textrm{otherwise}
\end{array}
\right. 
$$


**Part A**: Compute by hand the expected temperature in degrees Celsius on any given day. 

**Solution**: We have 

$$
E[X] = \int_{-\infty}^{\infty} x~f(x) dx = 
\int_{0}^{50} x~\frac{2}{5}\left[\left(\frac{x}{50}\right)^3 - \left(\frac{x}{50}\right)^4 \right] dx = 
\frac{2}{5}\int_{0}^{50} x~\left[\left(\frac{x}{50}\right)^3 - \left(\frac{x}{50}\right)^4 \right] dx = 
$$

$$
\frac{2}{5}\int_{0}^{50} \left[\frac{1}{50^3}x^4 - \frac{1}{50^4}x^5 \right] dx = 
\frac{2}{5}\left[\frac{50^5}{5\cdot 50^3} - \frac{50^6}{6\cdot 50^4}\right] = 
\frac{2}{5}\left[\frac{50^2}{5} - \frac{50^2}{6}\right] = 
\frac{2}{5}\cdot\frac{50^2}{30} = \frac{10\cdot 50}{15} = \frac{100}{3}^\circ \textrm{ C} 
$$

<p> </p> 

**Part B**: Use your result from **Part A** to compute the expected temperature in degrees Fahrenheit.  

**Solution**: We want to compute the expectation of $g(X)$ where $g$ converts from degrees Celsius to Fahrenheit.  Specifically, we have 

$$
g(X) = \frac{9}{5}X + 32 
$$

We then have 

$$
E[g(X)] = E\left[\frac{9}{5}X + 32\right] = \frac{9}{5}E[X] + 32 = \frac{9}{5}\cdot\frac{100}{3} + 32 = 92^\circ \textrm{ F}
$$